In [3]:
# Import necessary libraries
import csv
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Sample data representing air pressure readings for each drone
# Each row represents a drone, and each column represents a feature (e.g., microphone array readings)
data_Altura = []
data_ANWB = []
data_ATMOS = []
data_Autel_Evo = []
data_Phantom = []
name_dic = {'Altura.csv': data_Altura, 'ANWB.csv': data_ANWB, 'ATMOS.csv': data_ATMOS, 'Autel_Evo.csv': data_Autel_Evo, 'Phantom.csv': data_Phantom}

def datasets():
    data_lst = ['Altura.csv', 'ANWB.csv', 'ATMOS.csv', 'Autel_Evo.csv', 'Phantom.csv']
    for filename in data_lst:
        data = []
        with open(filename, "r") as file:
            csvreader = csv.reader(file)
            for row in csvreader:
                row1 = []
                for i,e in enumerate(row):
                    row1.append(float(e))
                data.append(row1)
        step = np.array(data)
        for i,row in enumerate(step.T):
            if i != 0 and i != 16 and  i !=20 and i != 40 and  i !=62 and  i !=63:
                name_dic[filename].append(row)

datasets()

In [25]:
def data_clean(drone, sample_percent):
    data_arr = np.array(name_dic[drone]).T
    total_time_points_no = len(data_arr)
    sample_size = int(total_time_points_no * sample_percent)
    X_data = np.empty((int(1/sample_size), sample_size))
    for i in range(0,int(1/sample_percent)-1):
        X = data_arr[sample_size*i:sample_size*(i+1)].T
        np.vstack((X_data,X))

    return X_data

sample_percent = 0.2
X_data = data_clean('Altura.csv', sample_percent)
print(X_data)




# freq[np.where(np.isclose(pxx, max(pxx)))[0]]



# Corresponding labels for each drone
labels = np.array([0, 1, 2, 3, 4])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create an SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions, normalize=True)

print("Accuracy:", accuracy)

[]


NameError: name 'data' is not defined